## New York City Taxi Fare Prediction
### Can you predict a rider's taxi fare?
<br>
<br>
This is one of the Kaggle Competition for biginner, alredy completed.<br>
Even it is very simple and straitforward for estimation, i thought it will be a good example to practice estimation problem from the scrath. <br>

You can refer to here for details, https://www.kaggle.com/c/new-york-city-taxi-fare-prediction <br>
This is a tensorflow code I made for your reference,
Your task is to create PyTorch code and run your own code, and compare it with other competitions.

If you have any questions on this implementaion, email [me](jychoi.ethan@gmail.com)

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

#!pip install -q tensorflow==2.0.0-beta1
import sys, os
import pandas as pd
import tensorflow as tf
import numpy as np

In [ ]:
# Make sure GPU is available on your machine
from tensorflow.python.client import device_lib
print(sys.version)
print("tf.version=", tf.__version__)
print(device_lib.list_local_devices())

## Read data using pandas
set path of the csv file containing the heart dataset.

In [ ]:
files = os.listdir('./')
files

### The train file is too large and only 10% of the samples are imported for this training.
```
# the original size of train data is 55423856, about 55M
df = df_train.sample(frac=0.1, replace=True, random_state=1)
print('The size of train data =', len(df))

df.to_csv('./train_s.csv')
```

### Please download pre-created train_s file which samples 10% of the original file randomly and it's test_s file for model evaluation
**train_s.csv file download** 
[train_s.csv](https://drive.google.com/uc?export=download&id=10Xnm58RpMlH2q1a5XwUoWvtbEkDV44Jg)
<br>**test_s.csv file download**
[test_s.csv](https://drive.google.com/uc?export=download&id=1Ej119VhEqt_IstdXgz3d-K_SDWevxghh)

Read the csv file using pandas.<br>
Instead of using original train.csv file, use smaller version, train_s.csv

In [ ]:
# train_file = './train.csv'
train_s_file = './train_s.csv'
test_s_file = './test_s.csv'
# original test file for submission
test_file = './test.csv'

### Let's read csv data into pandas dataframe
It will take some time to load as train.csv is a bit huge, please be patient 

In [ ]:
df_train = pd.read_csv(train_s_file, index_col=0)
df_test = pd.read_csv(test_s_file, index_col=0)
s_test = pd.read_csv(test_file, index_col=0)
print('The size of train data =', df_train.shape)
print('The size of test =', df_test.shape)
print('The size of test for submission =', s_test.shape)

print('\n', df_train.dtypes)

**Let's see what the data looks like**

In [ ]:
df_train.head(2)

### Data preprocess
- Drop columns not to be used for features, rows which has NaN values
- Remove outliers
- apply MinMaxScaler if possible

In [ ]:
# Let's look over train data if there are Nan in any of features
print(df_train.isnull().sum() , df_test.isnull().sum())

In [ ]:
# Remove missing values
df_train = df_train.dropna(how = 'any')

### From [the starter code](https://www.kaggle.com/dster/nyc-taxi-fare-starter-kernel-simple-linear-model) 

Add columns

```
# Given a dataframe, add two new features 'abs_diff_longitude' and
# 'abs_diff_latitude' reprensenting the "Manhattan vector" from
# the pickup location to the dropoff location.
def add_travel_vector_features(df):
    df['abs_diff_longitude'] = (df.dropoff_longitude - df.pickup_longitude).abs()
    df['abs_diff_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()

add_travel_vector_features(train_df)
```

In [ ]:
def add_travel_vector_features(df):
    df['abs_diff_longitude'] = (df.dropoff_longitude - df.pickup_longitude).abs()
    df['abs_diff_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()

# add longitude, latitude difference to column features
add_travel_vector_features(df_train)
add_travel_vector_features(df_test)
add_travel_vector_features(s_test)

### Filtering out outliers
From the starter code<br>
"We expect most of these values to be very small (likely between 0 and 1) since it should all be differences between GPS coordinates within one city. For reference, one degree of latitude is about 69 miles. However, we can see the dataset has extreme values which do not make sense. Let's remove those values from our training set. Based on the scatterplot, it looks like we can safely exclude values above 5"

In [ ]:
df_train = df_train[(df_train.abs_diff_longitude < 5.0) & (df_train.abs_diff_latitude < 5.0)]
print(df_train.shape)

Let's find out min, max values of longitude, latitude in test data set

In [ ]:
def location_boundary(df):
    """
    Find min, max values of logitude, latitude in df
    
    Return
        (longitude_min, longitude_max), (latitude_min, latitude_max)
    """
    
    log_min = min(df.pickup_longitude.min(), df.dropoff_longitude.min())
    log_max = min(df.pickup_longitude.max(), df.dropoff_longitude.max())
    
    lat_min = min(df.pickup_latitude.min(), df.dropoff_latitude.min())
    lat_max = min(df.pickup_latitude.max(), df.dropoff_latitude.max())
    
    return (log_min, log_max), (lat_min, lat_max)

In [ ]:
(lon_min, lon_max), (lat_min, lat_max) = location_boundary(s_test)
print(lon_min, lon_max, lat_min, lat_max)

**Let's remove train data out of bounding box of test set**

In [ ]:
df_train = \
    df_train[(df_train.pickup_longitude >= lon_min) & (df_train.pickup_longitude <= lon_max) &
    (df_train.dropoff_longitude >= lon_min) & (df_train.dropoff_longitude <= lon_max) &
    (df_train.pickup_latitude >= lat_min) & (df_train.pickup_latitude <= lat_max) &
    (df_train.dropoff_latitude >= lat_min) & (df_train.dropoff_latitude <= lat_max)]

print(df_train.shape)

**Most of cases, passenger_count is lower than 6, let's regard as outliers if it is more than 6, remove 0 passenger count as well**

Already applied to test_s set

In [ ]:
s_test.groupby('passenger_count').size()

In [ ]:
df_train = df_train[(df_train.passenger_count <= 6) & (df_train.passenger_count > 0)]
print(df_train.shape)

**Let's remove fare_amount lower than 2.5**

In [ ]:
df_train = df_train[ (df_train.fare_amount >= 2.5) & (df_train.fare_amount < 100)]
print(df_train.shape)

**Add pickup time inforation as input features**

In [ ]:
df_train['pickup_datetime'] = pd.to_datetime(df_train['pickup_datetime'],format='%Y-%m-%d %H:%M:%S UTC')

In [ ]:
def pickup_time_featues(data):
    """
    Insert year, month, day, day of week, hour, minute information 
    from pickup_datatime column
    """
    
    data['Year'] = data['pickup_datetime'].dt.year
    data['Month'] = data['pickup_datetime'].dt.month
    data['Date'] = data['pickup_datetime'].dt.day
    data['DayofWeek'] = data['pickup_datetime'].dt.dayofweek
    data['Hour'] = data['pickup_datetime'].dt.hour
    data['Minute'] = data['pickup_datetime'].dt.minute
    
    return data

In [ ]:
df_train = pickup_time_featues(df_train)
df_train.head(1)

In [ ]:
df_test['pickup_datetime'] = pd.to_datetime(df_test['pickup_datetime'],format='%Y-%m-%d %H:%M:%S UTC')
s_test['pickup_datetime'] = pd.to_datetime(s_test['pickup_datetime'],format='%Y-%m-%d %H:%M:%S UTC')
df_test = pickup_time_featues(df_test)
s_test = pickup_time_featues(s_test)

In [ ]:
## Let's see the final size of data again now
print('The shape of train data =', df_train.shape)
print('The shape of test =', df_test.shape)
print('The size of test for submission =', s_test.shape)

### Set columns to drop

In [ ]:
def preprocess_data(df, cols_to_drop='', label='fare_amount'):
    """
    Returns df_x(features), df_y(labels)
    """
    drop_cols = []
    for col in cols_to_drop:
        if col in df_train:
            drop_cols.append(col)

    df_x = df.drop(columns=drop_cols)

    if label is not None:
        df_y = df_x.pop(label)
        return df_x, df_y
    else:
        return df_x

In [ ]:
df_train.columns.values

In [ ]:
# columns to drop in input features
# simply not to use pickup_datetime as a feature for estimation here.
COLUMNS_TO_DROP = ['pickup_datetime']

In [ ]:
train_x, train_y = preprocess_data(df_train, COLUMNS_TO_DROP, 'fare_amount')
test_x, test_y = preprocess_data(df_test, COLUMNS_TO_DROP, 'fare_amount')
stest_x = preprocess_data(s_test, COLUMNS_TO_DROP, None)

In [ ]:
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape, stest_x.shape)

In [ ]:
train_x.head(2)

### Let's find out min, max values of each columns in train, test set

In [ ]:
def _MinMax(dframes):
    """
    Find out min max values of each columns among data frame listed in dframes
    """
    s_min = dframes[0].min(axis=0)
    s_max = dframes[0].max(axis=0)
    
    for df in dframes[1:]:
        d_min =  df.min(axis=0)
        d_max =  df.max(axis=0)
        s_min = s_min.combine(d_min, min)
        s_max = s_max.combine(d_max, max)
    return s_min, s_max

In [ ]:
x_min, x_max = _MinMax([train_x, test_x, stest_x])

In [ ]:
print(x_min, '\n', x_max)

In [ ]:
def _MinMaxScaler(X, s_min, s_max, feature_range=(0, 1)):
    
    min, max = feature_range
    
    episilon = 1e-10
    X_std = (X - s_min) / (s_max - s_min + episilon)
    X_scaled = X_std * (max - min) + min
    
    return X_scaled

### Apply MinMaxScaler to train and test dataframe
(This is optional)

In [ ]:
train_x = _MinMaxScaler(train_x, x_min, x_max)
test_x = _MinMaxScaler(test_x, x_min, x_max)
stest_x = _MinMaxScaler(stest_x, x_min, x_max)

In [ ]:
train_x.head(2)

## Load data using `tf.data.Dataset`

Use tf.data.Dataset.from_tensor_slices to read the values from a pandas dataframe.

One of the advantages of using tf.data.Dataset is it allows you to write simple, highly efficient data pipelines. Read the loading data guide to find out more.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((train_x.values, train_y.values))

For reading<br>
```for feat, targ in dataset.take(2):
       print ('Features: {}, Target: {}'.format(feat, targ))```

Shuffle and batch the dataset.

In [ ]:
BATCH_SIZE = 128

In [ ]:
train_dataset = dataset.shuffle(len(train_x)).batch(BATCH_SIZE)
print(train_dataset)
_, input_size = train_x.shape

## Create and train a model

In [ ]:
def get_compiled_model(hidden_units, activation='relu', lr=1e-3, l2_scale=1e-4):
    model = tf.keras.Sequential()
    
    for units in hidden_units:
        model.add(tf.keras.layers.Dense(units, \
                activation=activation, kernel_regularizer=tf.keras.regularizers.l2(l2_scale)))
    
    model.add(tf.keras.layers.Dense(1, activation='relu', \
                        kernel_regularizer=tf.keras.regularizers.l2(l2_scale)))
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    #optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr, momentum=0.9)
    
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    
    return model

In [ ]:
model = get_compiled_model([input_size, 13, 13], lr=1e-3, l2_scale=1e-4)

## Training

In [ ]:
# Just run 2 times as a trial
model.fit(train_dataset, epochs=2)

In [ ]:
model.summary()

## Model Evaluation

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_x.values, test_y.values))
test_dataset = test_dataset.batch(BATCH_SIZE)

**Evaluate how your model is accurate for prediction using mean absolute error**

In [ ]:
test_y_mse, test_y_mae = model.evaluate(test_dataset)
print('\nTest Loss {:.4f}, Test Mean Absolute Error {:.4f}'.format(test_y_mse, test_y_mae))

In [ ]:
test_y_predict = model.predict(test_dataset)

In [ ]:
print(test_y_predict.shape)
test_y_predict[:10, 0]

In [ ]:
test_y = test_y.values.reshape(-1,1)

In [ ]:
taxi_fare_predict = np.concatenate((test_y, test_y_predict), axis=1)

In [ ]:
taxi_fare = pd.DataFrame(taxi_fare_predict, columns=['fare_amount', 'prediction'])
taxi_fare.describe()

In [ ]:
y = abs(taxi_fare.fare_amount - taxi_fare.prediction)
y.describe()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(6, 4))
n, bins, patches = ax.hist(y, bins=y.size, linewidth=2, density=True, \
            histtype='step', cumulative=True)
ax.grid(True)
ax.set_title('New York Taxi Fare Prediction Error CDF')
ax.set_xlabel('fare amount error')
ax.set_ylabel('Likelihood of occurrence')
plt.show()

## Model predict for final test set

In [ ]:
stest_dataset = tf.data.Dataset.from_tensor_slices((stest_x.values))
stest_dataset = stest_dataset.batch(BATCH_SIZE)

In [ ]:
sfare_predict = model.predict(stest_dataset)

In [ ]:
print('the size of fare_predict=', sfare_predict.shape)

In [ ]:
sfare_predict = sfare_predict.reshape(-1)
key = s_test.index.values
print(key.shape, sfare_predict.shape)

In [ ]:
# Write the predictions to a CSV file which we can submit to the competition.
submission = pd.DataFrame(
    {'key': s_test.index, 'fare_amount': sfare_predict},
    columns = ['key', 'fare_amount'])
submission.to_csv('submission.csv', index = False)